In [32]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import sys
sys.path.append('..')

from config import Config
from vision_model import VisionEncoder

device = torch.device("mps")

In [34]:
img = torch.rand(size=(2, Config.img_size, Config.img_size, 3), device=device)
img.size()

torch.Size([2, 448, 448, 3])

In [35]:
config = Config()
vis_encoder = VisionEncoder(config=config)
vis_encoder = vis_encoder.to(device)
img_encod = vis_encoder(img)
img_encod.size()

torch.Size([2, 1960])

In [6]:
vis_encoder.pos_embeddings(torch.arange(0, 1, dtype=torch.int, device=device))

tensor([[-0.0854,  0.3218,  0.2398, -0.1488,  0.2133,  0.1039, -0.2728,  0.1281,
          0.7001,  0.4149]], device='mps:0', grad_fn=<EmbeddingBackward0>)